# 07 - DecisionTree

Realizando a predição com DecisionTree. Lembrando que a acurácia para os dados de treino precisa ser superior a 78% para que seja considerada válida para testarmos com os dados de teste.

## Preparando o ambiente

In [1]:
#!pip install graphviz
#!conda install python-graphviz

In [70]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, GroupKFold, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler
import graphviz
from IPython.display import display

print(f'GraphViz: {graphviz.__version__}')

GraphViz: 0.17


In [3]:
sns.set(rc={'figure.figsize':(18, 12)})

In [4]:
def exibir_arvore(modelo):
  dot_data = export_graphviz(modelo, 
                  out_file=None,
                  filled=True,
                  rounded=True)
  display(graphviz.Source(dot_data))

## Carregando os dados

### Carregando dados de treino

In [5]:
titanic = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed/train_processed_byage_ag.csv')
del titanic['Unnamed: 0']
titanic.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,male,22,1,0,SC,1,jovem_adulto,True,False
1,0,3,male,22,0,0,SC,0,jovem_adulto,False,False
2,0,3,male,22,0,0,SC,0,jovem_adulto,False,False
3,0,3,male,22,0,0,SC,0,jovem_adulto,False,False
4,0,3,male,22,0,0,SC,0,jovem_adulto,False,False


In [6]:
sex_map = {
    'male': 0,
    'female': 1
}
titanic['Sex'] = titanic['Sex'].map(sex_map)
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,0,22,1,0,SC,1,jovem_adulto,True,False
1,0,3,0,22,0,0,SC,0,jovem_adulto,False,False


In [7]:
boolean_map = {
    True: 1,
    False: 0
}
titanic['acompanhado'] = titanic['acompanhado'].map(boolean_map)
titanic['possui_cabine'] = titanic['possui_cabine'].map(boolean_map)
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,0,22,1,0,SC,1,jovem_adulto,1,0
1,0,3,0,22,0,0,SC,0,jovem_adulto,0,0


In [8]:
titanic['Floor'].unique()

array(['SC', 'C', 'E', 'B', 'A', 'D', 'G', 'F', 'T'], dtype=object)

In [9]:
floor_map = {
    'SC': 0,
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
    'T': 8
}
titanic['Floor'] = titanic['Floor'].map(floor_map)
titanic.isnull().sum()

Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Floor            0
Relateds         0
faixa_etaria     0
acompanhado      0
possui_cabine    0
dtype: int64

In [10]:
faixa_etaria_map = {
    'jovem_adulto': 1,
    'adulto_idoso': 2,
    'crianca_adolescente': 0
}
titanic['faixa_etaria'] = titanic['faixa_etaria'].map(faixa_etaria_map)
titanic.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,0,3,0,22,1,0,0,1,1,1,0
1,0,3,0,22,0,0,0,0,1,0,0


In [11]:
treino_1 = titanic
treino_1.sample(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
15,0,3,0,22,8,2,0,10,1,1,0
215,0,1,0,39,0,0,1,0,1,0,1


### Carregando dados de teste

In [12]:
teste = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed/test_processed.csv')
del teste['Unnamed: 0']

teste['acompanhado'] = teste['acompanhado'].map(boolean_map)
teste['possui_cabine'] = teste['possui_cabine'].map(boolean_map)
teste['Floor'] = teste['Floor'].map(floor_map)
teste['Sex'] = teste['Sex'].map(sex_map)
teste['faixa_etaria'] = teste['faixa_etaria'].map(faixa_etaria_map)

Xvalidacao = teste.drop(columns=['PassengerId'])
Xvalidacao.head(5)

,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,possui_cabine,acompanhado,faixa_etaria
0,3,0,34,0,0,0,0,0,0,1
1,3,1,47,1,0,0,1,0,1,2
2,2,0,62,0,0,0,0,0,0,2
3,3,0,27,0,0,0,0,0,0,1
4,3,1,22,1,1,0,2,0,1,1


### Separando treino e teste

In [13]:
SEED = 5
np.random.seed(SEED)

In [14]:
X = treino_1.drop(columns=['Survived'])
y = treino_1['Survived']

In [15]:
treino_x, teste_x, treino_y, teste_y = train_test_split(X, y, test_size = 0.2, stratify = y)

In [16]:
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 712 elementos e testaremos com 179 elementos


In [31]:
cv = GroupKFold(n_splits = 10)

## Execução da Árvore de decisão

In [17]:
espaco_de_parametros = {
    "criterion": ["gini", "entropy"],
    "max_depth" : [2, 3, 5, 7, 9, 11, 13],
    "min_samples_split": [2, 8, 16, 32],
    "min_samples_leaf": [1, 2, 8, 16, 32]    
}

In [18]:
busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)

In [19]:
busca.fit(treino_x, treino_y)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 7, 9, 11, 13],
                         'min_samples_leaf': [1, 2, 8, 16, 32],
                         'min_samples_split': [2, 8, 16, 32]})

In [20]:
resultados = pd.DataFrame(busca.cv_results_)
resultados.sample(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
251,0.002989,0.001091,0.001596,0.000797,entropy,11,8,32,"{'criterion': 'entropy', 'max_depth': 11, 'min...",0.797203,0.776224,0.732394,0.781690,0.830986,0.783699,0.031970,193
264,0.001595,0.000488,0.000797,0.000399,entropy,13,2,2,"{'criterion': 'entropy', 'max_depth': 13, 'min...",0.790210,0.797203,0.816901,0.746479,0.823944,0.794947,0.027201,68


In [21]:
print(busca.best_params_)
print(busca.best_score_ * 100)

{'criterion': 'gini', 'max_depth': 13, 'min_samples_leaf': 2, 'min_samples_split': 8}
81.60543681670444


In [22]:
busca.best_estimator_.score(teste_x, teste_y) * 100

84.35754189944134

A acurácia obtida de 86.2% já foi melhor do que o classificador manual, mas ainda não satisfatoriamente melhor.

## Normalizando os dados

Realizando os testes normalizando através de: `StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler`

### `StandardScaler`

In [23]:
scaler = StandardScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

In [24]:
busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)

{'criterion': 'gini', 'max_depth': 13, 'min_samples_leaf': 2, 'min_samples_split': 8}
82.02797202797203


In [25]:
busca.best_estimator_.score(n_teste_x, teste_y) * 100

84.35754189944134

`MaxAbsScaler`

In [26]:
scaler = MaxAbsScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)
print(busca.best_estimator_.score(n_teste_x, teste_y) * 100)

{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 8}
81.32670146754654
86.03351955307262


In [27]:
scaler = MinMaxScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)
print(busca.best_estimator_.score(n_teste_x, teste_y) * 100)

{'criterion': 'gini', 'max_depth': 13, 'min_samples_leaf': 2, 'min_samples_split': 8}
81.74628188712695
84.35754189944134


In [28]:
scaler = Normalizer()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)
print(busca.best_estimator_.score(n_teste_x, teste_y) * 100)

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 32}
79.35585541219343
79.3296089385475


In [29]:
scaler = RobustScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros)
busca.fit(n_treino_x, treino_y)
print(busca.best_params_)
print(busca.best_score_ * 100)
print(busca.best_estimator_.score(n_teste_x, teste_y) * 100)

{'criterion': 'gini', 'max_depth': 11, 'min_samples_leaf': 2, 'min_samples_split': 8}
82.16980202895697
84.91620111731844


Mesmo com a normalização, a acurácia se manteve em **86.0%**

## Testando com `GroupCV`

In [35]:
X

,Pclass,Sex,Age,SibSp,Parch,Floor,Relateds,faixa_etaria,acompanhado,possui_cabine
0,3,0,22,1,0,0,1,1,1,0
1,3,0,22,0,0,0,0,1,0,0
2,3,0,22,0,0,0,0,1,0,0
3,3,0,22,0,0,0,0,1,0,0
4,3,0,22,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
886,1,1,53,2,0,3,2,2,1,1
887,2,0,57,0,0,0,0,2,0,0
888,2,1,57,0,0,5,0,2,0,1
889,1,0,80,0,0,1,0,2,0,1


In [38]:
scaler = StandardScaler()
scaler.fit(X)
n_x = scaler.transform(X)

busca = GridSearchCV(DecisionTreeClassifier(), espaco_de_parametros, cv = cv)
busca.fit(n_x, y, groups=X.Age)
print(busca.best_params_)
print(busca.best_score_ * 100)

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
81.59122120077174


A acurácia parece ter piorado utilizando o cross validate.

## Busca em grid manual

In [62]:
def dt_grid_search(treino_x, treino_y, teste_x, teste_y):
    criterion = ["gini", "entropy"]
    max_depth = [2, 3, 5, 7, 9, 11, 13]
    min_samples_split = [2, 8, 16, 32]
    min_samples_leaf = [1, 2, 8, 16, 32]
    resultados = []

    for c in criterion:
        for md in max_depth:
            for ms in min_samples_split:
                for ml in min_samples_leaf:
                    modelo = DecisionTreeClassifier(criterion = c, max_depth = md, min_samples_split = ms, min_samples_leaf = ml)
                    modelo.fit(treino_x, treino_y)
                    score = modelo.score(teste_x, teste_y)
                    resultados.append({
                        "criterion": c,
                        "max_depth" : md,
                        "min_samples_split": ms,
                        "min_samples_leaf": ml,
                        "score": score
                    })

    return pd.DataFrame(resultados)
    
resultados = dt_grid_search(treino_x, treino_y, teste_x, teste_y).sort_values(by='score', ascending=False)
resultados

,criterion,max_depth,min_samples_split,min_samples_leaf,score
61,gini,7,2,2,0.860335
70,gini,7,16,1,0.860335
71,gini,7,16,2,0.860335
66,gini,7,8,2,0.860335
60,gini,7,2,1,0.854749
...,...,...,...,...,...
145,entropy,2,8,1,0.782123
144,entropy,2,2,32,0.782123
143,entropy,2,2,16,0.782123
142,entropy,2,2,8,0.782123


A melhor acurácia encontrada foi mesmo **86%**. Para o momento ficarei com essa e validarei com os dados de teste do Kaggle.

## Validando com dados de teste do Kaggle

In [57]:
best_params = resultados.sort_values(by=['score'], ascending=[False]).head(1).drop(columns=['score']).to_dict(orient='records')[0]
best_params

{'criterion': 'gini',
 'max_depth': 7,
 'min_samples_split': 16,
 'min_samples_leaf': 2}

In [58]:
melhor_modelo = DecisionTreeClassifier().set_params(**best_params)
melhor_modelo

DecisionTreeClassifier(max_depth=7, min_samples_leaf=2, min_samples_split=16)

In [59]:
melhor_modelo.fit(X, y)
predicoes = melhor_modelo.predict(Xvalidacao)

In [60]:
resultado = pd.DataFrame()
resultado['PassengerId'] = teste['PassengerId']
resultado['Survived'] = predicoes
resultado.sample(5)

,PassengerId,Survived
109,1001,0
231,1123,1
235,1127,0
264,1156,0
369,1261,0


In [61]:
resultado.to_csv('../../data/submissions/decision_tree.csv', index=False)

A acurácia no Kaggle foi de **76.5%**, ou seja, muito pouco melhor que o classificador manual.

## Scaler + Manual Grid Search

In [67]:
# `StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler`
scaler = RobustScaler()
scaler.fit(treino_x)
n_treino_x = scaler.transform(treino_x)
n_teste_x = scaler.transform(teste_x)

resultados = dt_grid_search(n_treino_x, treino_y, n_teste_x, teste_y).sort_values(by='score', ascending=False)
resultados

,criterion,max_depth,min_samples_split,min_samples_leaf,score
66,gini,7,8,2,0.860335
71,gini,7,16,2,0.860335
61,gini,7,2,2,0.860335
70,gini,7,16,1,0.860335
60,gini,7,2,1,0.849162
...,...,...,...,...,...
145,entropy,2,8,1,0.782123
144,entropy,2,2,32,0.782123
143,entropy,2,2,16,0.782123
142,entropy,2,2,8,0.782123


Os scalers não alteraram o resultado no treino então creio não valer apena tentar no Kaggle.

In [73]:
def dt_grid_search(X, y):
    criterion = ["gini", "entropy"]
    max_depth = [2, 3, 5, 7, 9, 11, 13]
    min_samples_split = [2, 8, 16, 32]
    min_samples_leaf = [1, 2, 8, 16, 32]
    resultados = []
    cv = GroupKFold(n_splits = 2)
    
    for c in criterion:
        for md in max_depth:
            for ms in min_samples_split:
                for ml in min_samples_leaf:
                    modelo = DecisionTreeClassifier(criterion = c, max_depth = md, min_samples_split = ms, min_samples_leaf = ml)
                    results = cross_validate(modelo, X, y, cv = cv, groups = X.faixa_etaria, return_train_score=True)
                    resultados.append({
                        "criterion": c,
                        "max_depth" : md,
                        "min_samples_split": ms,
                        "min_samples_leaf": ml,
                        "test_score": (results['test_score'][0] + results['test_score'][1]) / 2,
                        "train_score": (results['train_score'][0] + results['train_score'][1]) / 2
                    })

    return pd.DataFrame(resultados)

In [77]:
resultados = dt_grid_search(X, y).sort_values(by=['test_score', 'train_score'], ascending=[False, False])
resultados

,criterion,max_depth,min_samples_split,min_samples_leaf,test_score,train_score
245,entropy,11,8,1,0.805509,0.897946
205,entropy,7,8,1,0.803932,0.884959
200,entropy,7,2,1,0.801618,0.900207
225,entropy,9,8,1,0.801566,0.895580
265,entropy,13,8,1,0.799988,0.900312
...,...,...,...,...,...,...
37,gini,3,32,8,0.759869,0.842053
162,entropy,3,2,8,0.759869,0.842053
167,entropy,3,8,8,0.759869,0.842053
172,entropy,3,16,8,0.759869,0.842053


In [78]:
best_params = resultados.sort_values(by=['test_score'], ascending=[False]).head(1).drop(columns=['test_score', 'train_score']).to_dict(orient='records')[0]
best_params

{'criterion': 'entropy',
 'max_depth': 11,
 'min_samples_split': 8,
 'min_samples_leaf': 1}

In [79]:
melhor_modelo = DecisionTreeClassifier().set_params(**best_params)
melhor_modelo.fit(X, y)
predicoes = melhor_modelo.predict(Xvalidacao)
resultado = pd.DataFrame()
resultado['PassengerId'] = teste['PassengerId']
resultado['Survived'] = predicoes
resultado.to_csv('../../data/submissions/decision_tree.csv', index=False)

In [80]:
resultados = dt_grid_search(X, y).sort_values(by=['train_score', 'test_score'], ascending=[False, False])
resultados

,criterion,max_depth,min_samples_split,min_samples_leaf,test_score,train_score
120,gini,13,2,1,0.792943,0.941376
260,entropy,13,2,1,0.796149,0.939799
100,gini,11,2,1,0.784688,0.938222
240,entropy,11,2,1,0.797727,0.935067
80,gini,9,2,1,0.789788,0.928810
...,...,...,...,...,...,...
149,entropy,2,8,32,0.768387,0.780849
153,entropy,2,16,16,0.768387,0.780849
154,entropy,2,16,32,0.768387,0.780849
158,entropy,2,32,16,0.768387,0.780849


In [81]:
best_params = resultados.sort_values(by=['test_score'], ascending=[False]).head(1).drop(columns=['test_score', 'train_score']).to_dict(orient='records')[0]
melhor_modelo = DecisionTreeClassifier().set_params(**best_params)
melhor_modelo.fit(X, y)
predicoes = melhor_modelo.predict(Xvalidacao)
resultado = pd.DataFrame()
resultado['PassengerId'] = teste['PassengerId']
resultado['Survived'] = predicoes
resultado.to_csv('../../data/submissions/decision_tree.csv', index=False)

Os modelos retornados utilizando o CrossValidation apresentaram erro maior no kaggle (acurácia de 73.3% apenas). Isso indica que o problema da árvore não está na divisão dos dados de treino e teste.

### Tentativa final aleatória

In [82]:
clf_gini = DecisionTreeClassifier(criterion='gini',
                                 random_state=100,
                                 max_depth=5,
                                 min_samples_leaf=50,
                                 min_samples_split=50)
clf_gini.fit(X, y)
predicoes = clf_gini.predict(Xvalidacao)
resultado = pd.DataFrame()
resultado['PassengerId'] = teste['PassengerId']
resultado['Survived'] = predicoes
resultado.to_csv('../../data/submissions/decision_tree.csv', index=False)

## Conclusão

Para o formato atual dos dados a DecisionTree não apresentou resultados satisfatórios para o Kaggle. Alcançando o máximo de 76.5% de acurácia. Talvez uma alteração no pré processamento dos dados altere o resultado da árvore.

## Apêndice: Salvando os dados formatados para árvore

In [84]:
treino_1.to_csv('../../data/processed/train_processed_byage_ag_nm.csv', index=False)
teste.to_csv('../../data/processed/test_processed_byage_ag_nm.csv', index=False)